<center>
    <h1> INF335 - Tecnologías de Búsqueda Web   </h1>
    <h2> Tarea 1 </h2>
    <h3> Universidad Técnica Federico Santa Maria </h3>
    
</center>

_Marzo 2017_
<p>Profesor: Marcelo Mendoza</p>
 <p>Ayudante: Daniel Rivera</p>


<h2> Objetivos </h2>
<ul>
<li  style ="margin: 12px 0px; font-size:16px"> Implementar y analizar la herramienta de python NLTK (Natural Language Tookit) para trabajar con procesamiento de texto y lenguaje natural. </li>
<li style ="margin: 12px 0px;font-size:16px " > Estudiar e implementar las estructuras de datos adecuadas para representar un corpus, documentos y palabras con su categorización correspondiente.</li>
</ul>

<h2>Dataset : Amazon Fine Food Review</h2>

<p style="font-size:16px"> Para esta tarea se va a trabajar con el dataset de <i>“Amazon Fine Food Review”</i> el cual contiene más de 500.000 críticas de platos de comida y restaurants provenientes de Amazon. El archivo consiste en un .csv (“Comma Separate Values”) el cual contiene la siguiente estructura:</p>

<ol>
<li style ="margin: 5px 0px;"> <strong>Id</strong> - Id único de cada reseña</li>
<li style ="margin: 8px 0px;"><strong> ProductId</strong> - Id único que identifica el producto a analizar</li>
<li style ="margin: 8px 0px;"><strong>UserId</strong> - Id único que identifica al usuario</li>
<li style ="margin: 8px 0px;" ><strong>ProfileName</strong> - Nombre del usuario que realizó la reseña</li>
<li style ="margin: 8px 0px;"><strong>HelpfulnessNumerator</strong> -  número de usuarios que indicaron que encontraron esta crítica util</li>
<li style ="margin: 8px 0px;" ><strong>HelpfulnessDenominator</strong> número de usuarios que indicaron que encontraron esta crítica util -</li>
<li style ="margin: 8px 0px;" ><strong>Score</strong> - Rating, con valores entre 1 y 5 estrellas</li>
<li style ="margin: 8px 0px;" ><strong>Time</strong> - timestamp for the review</li>
<li style ="margin: 8px 0px;" ><strong>Summary</strong> - breve resumen de la reseña</li>
<li style ="margin: 8px 0px;"><strong>Text</strong> - string que contiene la reseña </li>
</ol>


<p style="font-size:16px" > <strong> Link Descarga Dataset: https://drive.google.com/open?id=0B1GNvIDVzwwLR2dwQVliRnBWMnM </strong>  </p> 

<p style="font-size:16px" > <b> Objetivo: </b> Extraiga del documento el item “Text” y generé un corpus , almacenando en un string todas las reseñas del dataset . Usará esta variable para realizar las siguientes etapas de preprocesamiento de texto.</p>

In [1]:
import nltk

In [2]:
import csv

with open('amazon-fine-foods/Reviews.csv', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    reseña = []
    for row in reader:
        reseña.append(row['Text'])

In [3]:
reseña[524000]

'If you are like me, chances are that you\'ve eaten a lot of Girl Scout cookies over the years, and if you are like most GSA cookie buyers, your favorites are the famous <a href="http://www.amazon.com/gp/product/B003D7TPZS">Girl Scout Thin Mint Cookies</a>.  If this is true, chances are you have found the occasional thin mint cookie that equaled them but not a thin mint cookie that bettered them,...<br /><br />until now.<br /><br />These <a href="http://www.amazon.com/gp/product/B004JGQ15Y">Oreo Peanut Butter Fudge Cr&egrave;mes</a>, also available in <a href="http://www.amazon.com/gp/product/B003XV1TAS">Oreo Regular Fudge Cr&egrave;mes</a>, <a href="http://www.amazon.com/gp/product/B003XV8XK2">Oreo Mint Fudge Cr&egrave;mes</a>, and <a href="http://www.amazon.com/gp/product/B004JGQ16I">Oreo Golden Oreo Fudge Cr&egrave;mes</a>, are IMHO BETTER tasting than the Girl Scout Thin Mint Cookies so while I might in future be persuaded to purchase fund raising thin mint cookies, they will be fo

<h2>Preprocesamiento:</h2>
# 1
<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > Si observa el corpus, se dará cuenta de que hay etiquetas <i>html</i> embebidas en algunas reseñas. Para eliminar estas etiquetas , use la libreria <i>Beautiful Soup</i> (link: https://www.crummy.com/software/BeautifulSoup/bs4/doc/).</p> </li>




In [4]:
from bs4 import BeautifulSoup

reseñas = []
for i in range(len(reseña)):
    markup = reseña[i]
    soup = BeautifulSoup(markup, 'lxml')
    reseña_sintag = soup.get_text()
    reseñas.append(reseña_sintag)

In [5]:
#reseñas is a list of reseñas after remove all tags (large 568454)
reseñas[524000]

"If you are like me, chances are that you've eaten a lot of Girl Scout cookies over the years, and if you are like most GSA cookie buyers, your favorites are the famous Girl Scout Thin Mint Cookies.  If this is true, chances are you have found the occasional thin mint cookie that equaled them but not a thin mint cookie that bettered them,...until now.These Oreo Peanut Butter Fudge Crèmes, also available in Oreo Regular Fudge Crèmes, Oreo Mint Fudge Crèmes, and Oreo Golden Oreo Fudge Crèmes, are IMHO BETTER tasting than the Girl Scout Thin Mint Cookies so while I might in future be persuaded to purchase fund raising thin mint cookies, they will be for other people.I'll be eating these!Note: As other reviewers have noted, this is not an entirely new concept for Oreo, which has offered limited editions of chocolate dipped whole sandwich cookies: Oreo Mint Fudge Covered Chocolate Sandwich Cookies, Oreo Pure Milk Chocolate Covered Mint Sandwich Cookies, and Oreo White Fudge Covered Chocolat

<p> A continuación, guardaremos el estado del corpus utilizando la librería Pickle </p>

In [6]:
#Using Pickle for save de state of corpus
import pickle

pickle_out = open("corpus.pickle", "wb")
pickle.dump(reseñas, pickle_out)
pickle_out.close()

<p> Corpus.pickle es una lista con todas las reseñas luego de remover las etiquetas html. </p>

In [8]:
import pickle

pickle_in = open("corpus.pickle", "rb")

reseñas = pickle.load(pickle_in)

<p> Luego, procederemos a dejar el corpus como una lista de listas de sentencias </p>

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize

reseña_sent_token = []

for i in range(len(reseñas)):
    reseña_sent_token.append(sent_tokenize(reseñas[i]))

In [11]:
#reseña_sent_token is a list of list that contain a tokenize sentences
reseña_sent_token[0]

['I have bought several of the Vitality canned dog food products and have found them all to be of good quality.',
 'The product looks more like a stew than a processed meat and it smells better.',
 'My Labrador is finicky and she appreciates this product better than  most.']

# 2
<li style ="margin: 12px 0px;">
<p style="font-size:16px"> Convierta el corpus , de modo que solo existan minúsculas (<code>lowercase</code>). </p></li>

In [16]:
corpus_sent_tokenize = []

for i in range(len(reseña_sent_token)):
    for j in range(len(reseña_sent_token[i])):
        corpus_sent_tokenize.append(reseña_sent_token[i][j].lower())

In [17]:
corpus_sent_tokenize[3] #corpus is a list with all sentences

'product arrived labeled as jumbo salted peanuts...the peanuts were actually small sized unsalted.'

In [16]:
#save corpus_sent_tokenize for sentimental analisys 
#Using Pickle for save de state of corpus
import pickle

pickle_out = open("corpus_sent_tokenize.pickle", "wb")
pickle.dump(corpus, pickle_out)
pickle_out.close()

## Correr desde acá para Sentymental Analysis

In [24]:
import pickle

pickle_in = open("corpus_sent_tokenize.pickle", "rb")

corpus_sent_tokenize = pickle.load(pickle_in)

<p> Tokenizando a nivel de palabra... </p>

In [18]:
from nltk.tokenize import word_tokenize

corpus_word_tokenize = []

for i in range(len(corpus_sent_tokenize)):
    corpus_word_tokenize.append(word_tokenize(corpus_sent_tokenize[i]))

# 3 
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando la lista de stopwords ortorgada por nltk, elimine aquellas palabras que sean clasificadas como stopwords, es decir, aquellas palabras que poseen poco contexto léxico y no otorgan información relevante. </p></li>

In [19]:
from nltk.corpus import stopwords

stop_words =set(stopwords.words("english"))
stop_words.update(['.','%','``', "''", '-', '--','...', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','>','<','$']) 

corpus_filtered = []

for i in corpus_word_tokenize:
    for j in i:
        if j not in stop_words:
            corpus_filtered.append(j)

In [20]:
import pickle

corpus = open("corpus_word_tokenize.pickle", "wb")
pickle.dump(corpus_filtered, corpus)
corpus.close()

## Desde acá comenzamos con el corpus ya tokenizado a nivel de word y luego de hacer el filtro de stopwords

In [21]:
import pickle

pickle_in = open("corpus_word_tokenize.pickle", "rb")
corpus = pickle.load(pickle_in)

# 4
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Elimine las palabras que aparezcan en el corpus con una frecuencia inferior a un umbral definido (ejemplo : inferior a 3) ( para ello, es recomendable determinar previo la frecuencia de cada término usando un diccionario). </p></li>

In [22]:
#corpus_filterbyfreq contain the words with frequency > 5 
corpus_filterbyfreq = []

#diccionario de palabras
diccionario = {}

for i in set(corpus):
    diccionario[i] = 0

for i in corpus:
    diccionario[i]+= 1

In [23]:
for i in corpus:
    if(diccionario[i]>10):
        corpus_filterbyfreq.append(i)

In [24]:
print(len(corpus))
print(len(corpus_filterbyfreq))
print(corpus_filterbyfreq[0:100])

23633518
23091889
['bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'product', 'looks', 'like', 'stew', 'processed', 'meat', 'smells', 'better', 'labrador', 'finicky', 'appreciates', 'product', 'better', 'product', 'arrived', 'labeled', 'jumbo', 'salted', 'peanuts', 'peanuts', 'actually', 'small', 'sized', 'unsalted', 'sure', 'error', 'vendor', 'intended', 'represent', 'product', 'jumbo', 'confection', 'around', 'centuries', 'light', 'pillowy', 'citrus', 'gelatin', 'nuts', 'case', 'filberts', 'cut', 'tiny', 'squares', 'liberally', 'coated', 'powdered', 'sugar', 'tiny', 'mouthful', 'heaven', 'chewy', 'flavorful', 'highly', 'recommend', 'yummy', 'treat', 'familiar', 'story', 'lewis', 'lion', 'witch', 'wardrobe', 'treat', 'edmund', 'selling', 'brother', 'sisters', 'witch', 'looking', 'secret', 'ingredient', 'robitussin', 'believe', 'found', 'got', 'addition', 'root', 'beer', 'extract', 'ordered', 'good', 'made', 'cherry', 'soda', 'flavor', 

## esto tarda mucho...
# 5
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando nltk, determine los Top-30 collocations mas relevantes del corpus, usando Bigramas .Implemente la función <code>BigramAssocMeasures()</code> y <code>BigramCollocationFinder.from_words()</code>. Recuerde que para este punto el corpus debe estar tokenizado. (mirar documentación). </p></li>

In [26]:
import nltk
from nltk.collocations import * 

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(corpus_filterbyfreq, 5)
print('Printing Top 30 collocations: ')
print(finder.nbest(bigram_measures.likelihood_ratio,30))

Printing Top 30 collocations: 
[('ca', "n't"), ('peanut', 'butter'), ('highly', 'recommend'), ('gluten', 'free'), ('grocery', 'store'), ('wo', "n't"), ('subscribe', 'save'), ('green', 'tea'), ("'ve", 'tried'), ('best', 'ever'), ('local', 'store'), ('year', 'old'), ('earl', 'grey'), ('dog', 'food'), ('cat', 'food'), ('would', 'recommend'), ('dark', 'chocolate'), ('olive', 'oil'), ('years', 'ago'), ('tea', 'tea'), ('cup', 'coffee'), ('tastes', 'like'), ('green', 'mountain'), ('much', 'better'), ('expiration', 'date'), ('coconut', 'oil'), ("'m", 'sure'), ('local', 'grocery'), ('science', 'diet'), ('sea', 'salt')]


# 6
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando la libreria incorporada en nltk, implemente Stanford POS tagger para categorizar y obtener los tags de cada token del corpus usando Part-Of-Speech Tagger (POSTagger). </p></li>

In [27]:
len(corpus_filterbyfreq)

23091889

In [28]:
import os
java_path="C:/Program Files/Java/jdk1.8.0_121/bin/java.exe"
os.environ['JAVAHOME'] = java_path

nltk.internals.config_java("C:/Program Files/Java/jdk1.8.0_121/bin/")
jar = '../stanford-postagger-full-2016-10-31/stanford-postagger.jar'
model = '../stanford-postagger-full-2016-10-31/models/english-left3words-distsim.tagger'

[Found C:/Program Files/Java/jdk1.8.0_121/bin/: C:/Program Files/Java/jdk1.8.0_121/bin/java.exe]


In [ ]:
from nltk.tag import StanfordPOSTagger

corpus_primeros_100 = set(corpus[0:100])

pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')

corpus_pos_tags = pos_tagger.tag(corpus_primeros_100)

In [54]:
print(corpus_pos_tags)

[('meat', 'NN'), ('lewis', 'NN'), ('edmund', 'NN'), ('soda', 'NN'), ('vendor', 'NN'), ('medicinal', 'JJ'), ('product', 'NN'), ('peanuts', 'NNS'), ('bought', 'VBD'), ('robitussin', 'NN'), ('beer', 'NN'), ('liberally', 'RB'), ('actually', 'RB'), ('stew', 'NN'), ('citrus', 'NN'), ('brother', 'NN'), ('quality', 'NN'), ('represent', 'VBP'), ('sure', 'JJ'), ('small', 'JJ'), ('pillowy', 'NN'), ('addition', 'NN'), ('highly', 'RB'), ('looking', 'VBG'), ('ingredient', 'NN'), ('secret', 'JJ'), ('witch', 'NN'), ('cut', 'NN'), ('food', 'NN'), ('labeled', 'VBN'), ('seduces', 'VBZ'), ('confection', 'NN'), ('sized', 'VBN'), ('c.s', 'NNS'), ('ordered', 'VBD'), ('labrador', 'NN'), ('appreciates', 'VBZ'), ('heaven', 'NN'), ('around', 'IN'), ('sisters', 'NNS'), ('error', 'NN'), ('lion', 'NN'), ('arrived', 'VBD'), ('believe', 'VB'), ('got', 'VBD'), ('powdered', 'JJ'), ('root', 'NN'), ('smells', 'VBZ'), ('mouthful', 'JJ'), ('light', 'NN'), ('products', 'NNS'), ('vitality', 'NN'), ('nuts', 'NNS'), ('salted',

In [50]:
pos_tag = nltk.pos_tag(corpus_primeros_100)
print(pos_tag)

[('i highly recommend this yummy treat.', 'NN'), ('!', '.'), ('between me, my kids, and my husband, this lasted only two weeks!', 'NN'), ('the flavor is very medicinal.', 'JJ'), ("i got this for my mum who is not diabetic but needs to watch her sugar intake, and my father who simply chooses to limit unnecessary sugar intake - she's the one with the sweet tooth - they both loved these toffees, you would never guess that they're sugar-free and it's so great that you can eat them pretty much guilt free!", 'JJ'), ('i like to transfer them to a zip lock baggie so they stay fresh so i can take my time eating them.', 'NN'), ("right now i'm mostly just sprouting this so my cats can eat the grass.", 'NN'), ('if you are familiar with the story of c.s.', 'NN'), ('i love these twizzlers!', 'NN'), ('if you are looking for the secret ingredient in robitussin i believe i have found it.', 'NN'), ("we've bought these many times from amazon because we're government employees living overseas and can't ge

## 7
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > Usando la libreria incorporada en nltk, implemente Named Entity Recognition (NER) con Stanford NER Tagger. Analice y describa sus resultados. </p></li>

In [51]:
from nltk.tag import StanfordNERTagger

st = StanfordNERTagger('../stanford-ner-2016-10-31/classifiers/english.all.3class.distsim.crf.ser.gz', '../stanford-ner-2016-10-31/stanford-ner.jar', encoding='utf-8')

corpus_ner_tagger = st.tag(corpus_primeros_100)

In [52]:
print(corpus_ner_tagger)

[('i', 'O'), ('highly', 'O'), ('recommend', 'O'), ('this', 'O'), ('yummy', 'O'), ('treat.', 'O'), ('!', 'O'), ('between', 'O'), ('me,', 'O'), ('my', 'O'), ('kids,', 'O'), ('and', 'O'), ('my', 'O'), ('husband,', 'O'), ('this', 'O'), ('lasted', 'O'), ('only', 'O'), ('two', 'O'), ('weeks!', 'O'), ('the', 'O'), ('flavor', 'O'), ('is', 'O'), ('very', 'O'), ('medicinal.', 'O'), ('i', 'O'), ('got', 'O'), ('this', 'O'), ('for', 'O'), ('my', 'O'), ('mum', 'O'), ('who', 'O'), ('is', 'O'), ('not', 'O'), ('diabetic', 'O'), ('but', 'O'), ('needs', 'O'), ('to', 'O'), ('watch', 'O'), ('her', 'O'), ('sugar', 'O'), ('intake,', 'O'), ('and', 'O'), ('my', 'O'), ('father', 'O'), ('who', 'O'), ('simply', 'O'), ('chooses', 'O'), ('to', 'O'), ('limit', 'O'), ('unnecessary', 'O'), ('sugar', 'O'), ('intake', 'O'), ('-', 'O'), ("she's", 'O'), ('the', 'O'), ('one', 'O'), ('with', 'O'), ('the', 'O'), ('sweet', 'O'), ('tooth', 'O'), ('-', 'O'), ('they', 'O'), ('both', 'O'), ('loved', 'O'), ('these', 'O'), ('toffee

In [44]:
ner_tag = nltk.ne_chunk(pos_tag)
print(ner_tag)

(S
  meat/NN
  lewis/NN
  edmund/NN
  soda/NN
  vendor/NN
  medicinal/JJ
  product/NN
  peanuts/NNS
  bought/VBD
  taffy/JJ
  robitussin/NN
  beer/NN
  liberally/RB
  actually/RB
  stew/JJ
  citrus/NN
  brother/NN
  quality/NN
  represent/VBP
  sure/RB
  small/JJ
  pillowy/JJ
  addition/NN
  highly/RB
  looking/VBG
  ingredient/NN
  secret/JJ
  witch/NN
  cut/NN
  food/NN
  labeled/VBN
  confection/NN
  sized/VBN
  ordered/VBD
  labrador/JJ
  appreciates/NNS
  heaven/VBP
  around/IN
  sisters/NNS
  error/VBP
  lion/NN
  arrived/VBN
  believe/VBP
  got/VBN
  powdered/JJ
  root/NN
  smells/VBZ
  mouthful/JJ
  light/JJ
  products/NNS
  vitality/NN
  nuts/RB
  salted/VBN
  like/IN
  good/JJ
  intended/VBN
  unsalted/JJ
  squares/NNS
  finicky/JJ
  cherry/VBP
  sugar/JJ
  story/NN
  canned/VBN
  recommend/NN
  made/VBN
  gelatin/JJ
  several/JJ
  treat/NN
  case/NN
  found/VBD
  better/JJR
  dog/NN
  processed/VBD
  great/JJ
  coated/JJ
  wardrobe/NN
  flavor/NN
  flavorful/NN
  familiar/JJ

# 8
<li style ="margin: 12px 0px;">
<p style="font-size:16px" > <strong> Sentiment Analysis </strong>: Implemente usando la libreria <i>Vader</i> incorporada en nltk para analizar la polaridad del corpus ,determinar cada documento (para ello es necesario re-estructurar el corpus como un array de documentos, o sentencias):</p>
<ol>
<li> Tokenizar el corpus a nivel de sentencia (recuerde incorporar el preprocesamiento previo).</li>
<li> Para cada sentencias (reseña) , implemente Vader para determinar la polaridad.</li>
</ol>
</li>

In [55]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [57]:
sid = SentimentIntensityAnalyzer()
for sentence in corpus_sent_tokenize[0:100]:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

i have bought several of the vitality canned dog food products and have found them all to be of good quality.
compound: 0.6369, neg: 0.0, neu: 0.776, pos: 0.224, 
the product looks more like a stew than a processed meat and it smells better.
compound: 0.6901, neg: 0.0, neu: 0.659, pos: 0.341, 
my labrador is finicky and she appreciates this product better than  most.
compound: 0.7351, neg: 0.0, neu: 0.617, pos: 0.383, 
product arrived labeled as jumbo salted peanuts...the peanuts were actually small sized unsalted.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
not sure if this was an error or if the vendor intended to represent the product as "jumbo".
compound: -0.5664, neg: 0.226, neu: 0.774, pos: 0.0, 
this is a confection that has been around a few centuries.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
it is a light, pillowy citrus gelatin with nuts - in this case filberts.
compound: -0.3182, neg: 0.173, neu: 0.827, pos: 0.0, 
and it is cut into tiny squares and then liberally coa

<h2>Notas</h2>
<ul>
<li style ="margin: 12px 0px; font-size: 16px" >
Para varias etapas del preprocesamiento, usará diferentes librerias disponibles en Python. Se recomienda usar el instalador de paquetes <i>pip</i> ( link: https://pypi.python.org/pypi/pip ) .
</li>
<li style ="margin: 12px 0px; font-size: 16px" >
Algunos de estos pasos del preprocesamiento pueden demorar en compilar (en algunos casos sobre 45 min, dependiendo de la máquina), por lo que es recomendable ir guardando el estado del corpus su posterior uso. Para estos casos se recomienda usar la libreria <code> pickle </code> en python (link: https://docs.python.org/2/library/pickle.html ) 
</li>
</ul>



<h2>Documentación y Ejemplos</h2>
<ul>
<li style ="margin: 12px 0px;" >
Beautiful Soup :
https://www.crummy.com/software/BeautifulSoup/
</li>
<li style ="margin: 12px 0px;" >
Bigrams and Collocations:
http://www.nltk.org/howto/collocations.html
</li>
<li style ="margin: 12px 0px;" >
Stanford PoS Tagger :
http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford
</li>
<li>
Stanford 'Tagger' Link Download : https://nlp.stanford.edu/software/tagger.shtml#Download
</li>
<li style ="margin: 12px 0px;" >
Stanford Ner Tagger:
https://pythonprogramming.net/named-entity-recognition-stanford-ner-tagger/
</li>
<li style ="margin: 12px 0px;" >
Sentiment Analysis with Vader: 
http://www.nltk.org/howto/sentiment.html
</li>

</ul>

<h2>Instrucciones</h2>

<ol >
<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > El informe debe entregarse en un archivo jupyter notebook (diferente a este) con el código  implementado y los análisis correspondientes. El informe debe subirse en la plataforma oficial de moodle en formato comprimido (.zip) con el nombre <i>tarea1_rol.zip</i></p> </li>

<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > Todas las consultas serán atendidas por el canal de consultas de moodle. </i></p> </li>

<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > La fecha de entrega es el dia <strong>10 de Abril</strong> . Pasada esa fecha se descontaran 20 puntos por dia. </p> </li>


</ol>